### Multiple linear regression
<br>
$$y = b_{0} + b_{1}*x_{1}+ b_{2}*x_{2} + b_{3}*x_{3} + ... + b_{n}*x_{n}$$
<br>
$y$ - dependent variable (what we are trying to explain);

$x_{1},  x_{2},  x_{3}$ - multiple independent variables (experience, how many courses done, college, etc.);

### Assuptions of a Linear Regression (Check if these are true)
1. Linearity
2. Homoscedasticity
3. Multivariate normality
4. Independence of errors
5. Lack of multicollinearity

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('50_Startups.csv')
dataset.head(5)

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


<br>
<center> Profit = $b_{0}$ + $b_{1}*$R&D Spend + $b_{2}*$Administration + $b_{3}*$Marketing Spend + <strike> $b_{4}*$State</strike> </center> 

- but "State" is not number, so a dummy variable is needed;
- if there are 2 dummy variables, the first one is enough for the analysis, so we should include only the first one;
- dummy variable that equals to 0 will become the default value, it will be included in the constant $b_{0}$.
<br><br>
If we include both dummy variables: $$b_{4}*D_{1} + b_{5}*D_{2}$$
- we are duplicating a variable because $D_{2}=1-D_{1}$;
- there is <b>multicollinearity</b>: the phenomenon where one or several independent variables predict another;
- → the model cannot distinguish between the effects of $D_{2}$ and $D_{1}$ and it won't work properly;
- <b>dummy variable trap</b> → avoid by always omitting one dummy variable.

## Building a Model

There are so many columns $x_{1}, x_{2}, x_{3}...$ = potential predictors for our $y$ dependent variable.

→ Decide which ones to keep and which to throw out. Select the right variables.
<br>
Why not to take all variables? 
- It won't be a good model.
- We need to explain how certain variables predict the other, so better to keep only the important ones. 
<br>

### 5 methods of building models

1. All-in;
 
2. Backward elimination;
    
3. Forward selection;
    
4. Bidirectional elimination (2-4 or only 4 are sometimes called <i>Stepwise Regression</i>);
    
5. Score comparison.



<br>
I. <b>"All-in"</b> cases:

- prior knowledge;
- we have to, someone asks us to;
- preparing for Backward elimination.

<br>
II. <b>Backward elimination</b>:

- select a significance level to stay in the model (e.g. SL = 0.05);
- fit the full model with all the predictors;
- consider the predictor with the highest P-value, if P > SL, go to the next step, else go to FIN (Your model is ready);
- remove the predictor;
- fit the model without this variable;
- repeat steps 3-5 until FIN. 

<br>
III. <b>Forward selection</b>:

- select a significance level to enter the model (e.g. SL = 0.05);
- fit all simple regression models $y ~ x_{n}$, select the one with lowest P-value;
- keep this variable and fit all possible two-variable models with this variable + one extra predictor (one by one starting from the one with the lowest P-value);
- consider the predictor with the lowest P-value. If P < SL, go to step 3 (& add +1 variable);
- stop when P > SL, no other variable will be significant.

<br>
IV. <b>Bidirectional elimination</b>:

- select a significance level to enter the model (SLENTER = 0.05) and to stay in the model (SLSTAY = 0.05);
- perform the next step of Forward Selection (new variables must have P < SLENTER to enter);
- perform all steps of Backward elimination (old variables must have P < SLSTAY to stay;
- repeate steps 2-3;
- until no new variables can enter and no old variables can exit.

<br>
V. <b>Score comparison, All possible models</b>:

- Select a criterion of goodness of fit (e.g. Akaike criterion, $R^2$, etc.);
- Construct all possible regression models: $2^N-1$ total combinations;
- Select the one with the best criterion.

In [2]:
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
X = dataset.iloc[:, :-1].values #independent variables
y = dataset.iloc[:, 4].values #dependent variable

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
columntransformer = ColumnTransformer([("State", OneHotEncoder(), [3])], remainder = 'passthrough')
X = columntransformer.fit_transform(X)
X = X.astype(int)

In [56]:
# Avoiding dummy variable trap

X = X[:, 1:] # all rows, all columns except at index 0

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=0)

In [58]:
# Fit multiple linear regression on the training set

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [59]:
# Testing

y_pred = regressor.predict(X_test)

In [60]:
pd.DataFrame(y_test)

,0
0,103282.38
1,144259.40
2,146121.95
3,77798.83
4,191050.39
5,105008.31
6,81229.06
7,97483.56
8,110352.25
9,166187.94


In [61]:
pd.DataFrame(y_pred)

,0
0,103015.246468
1,132581.940627
2,132448.093974
3,71975.743956
4,178537.520079
5,116161.051969
6,67851.477613
7,98791.741122
8,113969.410046
9,167921.224161


In [62]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [63]:
X = np.append(arr=np.ones((50,1)).astype(int), values=X, axis=1) #add an intercept (constant) of 1 manually

In [64]:
X_opt = X[:, [0,1,2,3,4,5]] #will contain only significant independant variables

In [66]:
# Create new regressor, fit the model with all possible predictors

regressor_OLS = sm.OLS(endog=y, exdog=X_opt).fit()

In [67]:
# Choose the variable with the highest p-value and remove it

regressor_OLS.summary() # x2 - 0.990 - remove index 2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Thu, 11 Nov 2021   Prob (F-statistic):           1.34e-27
Time:                        16:02:13   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.855      7.281      0.000    3.63e+04     6.4e+04
x1           198.7542   3371.026      0.059      0.953   -6595.103    6992.611
x2           -42.0063   3256.058     -0.013      0.990   -6604.161    6520.148
x3             0.8060      0.046     17.368      0.000       0.712       0.900
x4            -0.0270      0.052     -0.517      0.608      -0.132       0.078
x5             0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.783   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.267
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [69]:
X_opt = X[:, [0,1,3,4,5]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary() # remove index 1 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.946
Method:                 Least Squares   F-statistic:                     217.2
Date:                Thu, 11 Nov 2021   Prob (F-statistic):           8.49e-29
Time:                        16:04:08   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1061.
Df Residuals:                      45   BIC:                             1070.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.011e+04   6647.901      7.537      0.000    3.67e+04    6.35e+04
x1           220.1847   2900.553      0.076      0.940   -5621.828    6062.197
x2             0.8060      0.046     17.606      0.000       0.714       0.898
x3            -0.0270      0.052     -0.523      0.604      -0.131       0.077
x4             0.0270      0.017      1.592      0.118      -0.007       0.061
==============================================================================
Omnibus:                       14.759   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.173
Skew:                          -0.948   Prob(JB):                     2.53e-05
Kurtosis:                       5.563   Cond. No.                     1.40e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [70]:
X_opt = X[:, [0,3,4,5]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary() # remove x2 index 4

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Thu, 11 Nov 2021   Prob (F-statistic):           4.53e-30
Time:                        16:04:46   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.012e+04   6572.384      7.626      0.000    3.69e+04    6.34e+04
x1             0.8057      0.045     17.846      0.000       0.715       0.897
x2            -0.0268      0.051     -0.526      0.602      -0.130       0.076
x3             0.0272      0.016      1.655      0.105      -0.006       0.060
==============================================================================
Omnibus:                       14.839   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.443
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.587   Cond. No.                     1.40e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [71]:
X_opt = X[:, [0,3,5]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary() # remove x2 index 5 - marketing spend

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Thu, 11 Nov 2021   Prob (F-statistic):           2.16e-31
Time:                        16:05:55   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.941     17.464      0.000    4.16e+04    5.24e+04
x1             0.7966      0.041     19.265      0.000       0.713       0.880
x2             0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.678   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.162
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [72]:
# The only significant variable - R&D spend

X_opt = X[:, [0,3]]
regressor_OLS = sm.OLS(endog=y, exog=X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.947
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     849.8
Date:                Thu, 11 Nov 2021   Prob (F-statistic):           3.50e-32
Time:                        16:08:13   Log-Likelihood:                -527.44
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      48   BIC:                             1063.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.903e+04   2537.900     19.320      0.000    4.39e+04    5.41e+04
x1             0.8543      0.029     29.151      0.000       0.795       0.913
==============================================================================
Omnibus:                       13.727   Durbin-Watson:                   1.116
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               18.538
Skew:                          -0.911   Prob(JB):                     9.43e-05
Kurtosis:                       5.361   Cond. No.                     1.65e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""